In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/summer-analytics-mid-hackathon/hacktest.csv
/kaggle/input/summer-analytics-mid-hackathon/hacktrain.csv


In [2]:
from sklearn.impute import KNNImputer,SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler,LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,accuracy_score
import numpy as np, pandas as pd


df = pd.read_csv('/kaggle/input/summer-analytics-mid-hackathon/hacktrain.csv')


ndvi_cols = [col for col in df.columns if col.endswith('_N')]
imputer = SimpleImputer(strategy="median")
df[ndvi_cols] = imputer.fit_transform(df[ndvi_cols])

df['mean'] = df[ndvi_cols].mean(axis=1)
df['median']= df[ndvi_cols].median(axis=1)
df['max'] = df[ndvi_cols].max(axis=1)
df['min'] = df[ndvi_cols].min(axis=1)
df['std'] = df[ndvi_cols].std(axis=1)
df['range'] = df['max'] - df['min']

X = df[['mean', 'max', 'min', 'std','range','median']]
y = df['class']


scaler=StandardScaler()
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df['class'])
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.11, random_state=None, stratify=y)

model = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=1000)
model.fit(X_train, y_train)


y_pred = model.predict(X_val)


accuracy = accuracy_score(y_val, y_pred)
report = classification_report(y_val, y_pred,zero_division=1)

print("Accuracy:", accuracy)
print("Classification Report:\n", report)


test_df = pd.read_csv('/kaggle/input/summer-analytics-mid-hackathon/hacktest.csv')

ndvi_cols = [col for col in test_df.columns if col.endswith('_N')]

test_df[ndvi_cols] = imputer.transform(test_df[ndvi_cols])  # use same imputer as training

test_df['mean'] = test_df[ndvi_cols].mean(axis=1)
test_df['median']=test_df[ndvi_cols].median(axis=1)
test_df['max'] = test_df[ndvi_cols].max(axis=1)
test_df['min'] = test_df[ndvi_cols].min(axis=1)
test_df['std'] = test_df[ndvi_cols].std(axis=1)
test_df['range'] = test_df['max'] - test_df['min']
scaler.fit(X_val)  
X_test = test_df[['mean', 'max', 'min', 'std','range','median']]
X_test_scaled = scaler.transform(X_test)  # use same scaler as training


y_test_pred_encoded = model.predict(X_test_scaled)
y_test_pred = label_encoder.inverse_transform(y_test_pred_encoded)


submission_df = pd.DataFrame({
    'ID': test_df['ID'],
    'class': y_test_pred
})

print(submission_df.head(60))

submission_df.to_csv("submission.csv", index=False)



Accuracy: 0.8363636363636363
Classification Report:
               precision    recall  f1-score   support

           0       0.45      0.11      0.18        92
           1       0.86      0.99      0.92       677
           2       0.33      0.09      0.14        22
           3       0.77      0.66      0.71        74
           4       1.00      0.00      0.00         3
           5       0.78      0.58      0.67        12

    accuracy                           0.84       880
   macro avg       0.70      0.41      0.44       880
weighted avg       0.79      0.84      0.80       880

    ID       class
0    1  impervious
1    2       grass
2    3     orchard
3    4     orchard
4    5     orchard
5    6  impervious
6    7  impervious
7    8  impervious
8    9       water
9   10  impervious
10  11     orchard
11  12  impervious
12  13     orchard
13  14     orchard
14  15       grass
15  16  impervious
16  17     orchard
17  18     orchard
18  19     orchard
19  20  impervious
20  2

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.11/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
